In [5]:
import secrets
import hashlib
import ecdsa
from ecdsa.util import sigencode_string
from ecdsa.curves import SECP256k1

#
# Local Simulation of CGGMP21 Threshold ECDSA Signing Protocol (Simplified)
#
# This script simulates the CGGMP21 protocol, using the 'ecdsa' library
# to handle the underlying elliptic curve cryptography. This simplifies the code
# by abstracting away manual point addition, scalar multiplication, and verification.
#
# DISCLAIMER: This remains a simulation for educational purposes and does not
# include ZKPs or secure channels, which are critical for a production system.
#

# --- 1. Curve and Helper Functions ---
# We now use the ecdsa library's pre-defined constants and functions.
CURVE = SECP256k1
G = CURVE.generator
N = CURVE.order

def inv(n, prime=N):
    """Calculates modular inverse. Still needed for our polynomial math."""
    return pow(n, prime - 2, prime)

# --- 2. Lagrange Coefficient Calculation ---
def compute_lagrange_coeff(party_ids, target_id):
    """
    Computes the Lagrange coefficient lambda_i for a given party i at x=0.
    """
    numerator = 1
    denominator = 1
    for j in party_ids:
        if j != target_id:
            numerator = (numerator * j) % N
            denominator = (denominator * (j - target_id)) % N
    return (numerator * inv(denominator, N)) % N

# --- 3. Signer Class Definition ---
class Signer:
    """Represents a single participant in the DKG and signing protocol."""
    def __init__(self, party_id, num_parties, threshold):
        self.id = party_id
        self.num_parties = num_parties
        self.threshold = threshold

        # DKG state
        self.poly_coeffs = None
        self.commitments = None  # Will now store ecdsa.ellipticcurve.Point objects
        self.received_shares = {}
        self.private_key_share = None
        self.public_key_share = None

        # Signing state
        self.k_share = None
        self.R_share = None

    def dkg_round_1_create_poly_and_commitments(self):
        """Each party creates its own secret polynomial and commitment."""
        self.poly_coeffs = [secrets.randbelow(N) for _ in range(self.threshold)]
        # Commitments are now calculated using the library's scalar multiplication
        self.commitments = [c * G for c in self.poly_coeffs]

    def dkg_round_2_get_share_for_party(self, party_j_id):
        """Calculates the share s_ij = f_i(j) for party j."""
        y = 0
        for coeff in reversed(self.poly_coeffs):
            y = (y * party_j_id + coeff) % N
        return y

    def dkg_round_3_verify_and_store_share(self, from_party_id, share, commitments):
        """Verify the received share s_ji against party j's commitment C_j."""
        # lhs = s_ji * G
        lhs = share * G
        # rhs = sum( (i^k * C_jk) for k in 0..t )
        rhs = ecdsa.ellipticcurve.INFINITY
        for k in range(self.threshold):
            term = pow(self.id, k, N) * commitments[k]
            rhs += term

        if lhs != rhs:
            raise ValueError(f"Party {self.id} received a bad share from Party {from_party_id}")
        self.received_shares[from_party_id] = share

    def dkg_round_4_compute_key_shares(self):
        """Compute final private key share x_i = sum(s_ji for j in 1..n)."""
        self.private_key_share = sum(self.received_shares.values()) % N
        self.public_key_share = self.private_key_share * G

    def sign_round_1_create_nonce_share(self):
        """Each signing party generates a secret nonce share k_i and a public R_i."""
        self.k_share = secrets.randbelow(N)
        self.R_share = self.k_share * G
        return self.R_share

# --- Main Simulation ---
if __name__ == '__main__':
    N_PARTIES = 5
    THRESHOLD = 3  # Requires 3 parties to sign.

    print(f"--- CGGMP21 Simulation (Simplified with ecdsa library) ---")
    print(f"Total Parties (n): {N_PARTIES}, Threshold (t+1): {THRESHOLD}\n")

    # === 1. Setup ===
    parties = [Signer(i, N_PARTIES, THRESHOLD) for i in range(1, N_PARTIES + 1)]

    # === 2. Distributed Key Generation (DKG) ===
    print("--- Phase 1: Distributed Key Generation ---")
    all_commitments = {}
    for p in parties:
        p.dkg_round_1_create_poly_and_commitments()
        all_commitments[p.id] = p.commitments
    print("All parties created polys and commitments.")

    for sender in parties:
        for receiver in parties:
            if sender.id != receiver.id:
                share = sender.dkg_round_2_get_share_for_party(receiver.id)
                receiver.dkg_round_3_verify_and_store_share(sender.id, share, all_commitments[sender.id])
    print("All parties exchanged and verified shares successfully.")

    for p in parties:
        self_share = p.dkg_round_2_get_share_for_party(p.id)
        p.received_shares[p.id] = self_share
        p.dkg_round_4_compute_key_shares()
    print("All parties computed their final private key shares (x_i).")

    # Aggregate the public key Y = sum(Y_i0)
    agg_public_key_point = ecdsa.ellipticcurve.INFINITY
    for p in parties:
        agg_public_key_point += p.commitments[0]

    # Create a VerifyingKey object for later use
    agg_verifying_key = ecdsa.VerifyingKey.from_public_point(agg_public_key_point, curve=CURVE)

    print("\nDKG Complete!")
    print(f"Aggregated Public Key (Y):\n  x: {hex(agg_public_key_point.x())}\n  y: {hex(agg_public_key_point.y())}\n")

    # === 3. Threshold Signing ===
    print("--- Phase 2: Threshold Signing ---")
    message = b"CGGMP21 is a cool protocol"  # Use bytes for hashing
    msg_hash = hashlib.sha256(message).digest()

    signing_parties_ids = list(range(1, THRESHOLD + 1))
    signing_parties = [p for p in parties if p.id in signing_parties_ids]
    print(f"Message: '{message.decode()}'")
    print(f"Signing with parties: {signing_parties_ids}\n")

    # Round 1: Create nonce shares and aggregate R
    R_agg = ecdsa.ellipticcurve.INFINITY
    for sp in signing_parties:
        R_agg += sp.sign_round_1_create_nonce_share()

    r = R_agg.x() % N
    if r == 0:
        raise RuntimeError("r is zero, must restart signing round")
    print(f"Aggregated R computed. Signature 'r' value is: {hex(r)}")

    # Round 2: Compute final signature 's'
    m_int = int.from_bytes(msg_hash, 'big')
    x_reconstructed = 0
    for sp in signing_parties:
        lagrange_coeff = compute_lagrange_coeff(signing_parties_ids, sp.id)
        x_reconstructed = (x_reconstructed + lagrange_coeff * sp.private_key_share) % N

    k_agg = sum(sp.k_share for sp in signing_parties) % N
    s = (inv(k_agg) * (m_int + r * x_reconstructed)) % N

    if s == 0:
        raise RuntimeError("s is zero, must restart signing round")

    # Encode the signature in the raw format the library expects
    final_signature_raw = sigencode_string(r, s, N)
    print(f"Final signature 's' value is: {hex(s)}\n")

    # === 4. Verification ===
    print("--- Phase 3: Verification ---")
    try:
        # Use the library's verification method
        is_valid = agg_verifying_key.verify_digest(final_signature_raw, msg_hash)
        print(f"Signature valid: {is_valid}")
    except ecdsa.BadSignatureError:
        is_valid = False
        print("Signature valid: False (BadSignatureError)")

    if is_valid:
        print("\nSUCCESS: The threshold signature was verified correctly with the aggregated public key.")
    else:
        print("\nFAILURE: The signature verification failed.")
